In [2]:
import os
import argparse
import pandas as pd
import numpy as np
import xgboost as xgb
from math import ceil
from math import sqrt
from math import acos
from operator import itemgetter
from itertools import product
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import gc
import seaborn as sns
from scipy import stats
from collections import defaultdict
from tqdm import tqdm
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe , space_eval
import time

In [3]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Training new models on chymotrypsis-immunopeptides

In [4]:
def load_data(vector_filename, ion_type):
    # Read file
    if vector_filename.split(".")[-1] == "pkl":
        vectors = pd.read_pickle(vector_filename)
    elif vector_filename.split(".")[-1] == "h5":
        # vectors = pd.read_hdf(vector_filename, key='table', stop=1000)
        vectors = pd.read_hdf(vector_filename, key="table")
    else:
        print("Unsuported feature vector format")
        exit(1)

    # Extract targets for given ion type
    target_names = list(vectors.columns[vectors.columns.str.contains("targets")])
    if not "targets_{}".format(ion_type) in target_names:
        print("Targets for {} could not be found in vector file.".format(ion_type))
        print("Vector file only contains these targets: {}".format(target_names))
        exit(1)

    targets = vectors.pop("targets_{}".format(ion_type))
    target_names.remove("targets_{}".format(ion_type))
    for n in target_names:
        vectors.pop(n)

    # Get psmids
    psmids = vectors.pop("psmid")

    return (vectors, targets, psmids)

In [5]:
def get_params_combinations(params):
    keys, values = zip(*params.items())
    combinations = [dict(zip(keys, v)) for v in product(*values)]
    return(combinations)

In [6]:
def get_best_params(df, params_grid):
    params = {}
    best = df[df['test-rmse-mean'] == df['test-rmse-mean'].min()]
    best_rmse = df['test-rmse-mean'].min()
    for p in params_grid.keys():
        params[p] = best[p].iloc[0]
    # num_boost_round = best['boosting-round'].iloc[0]
    return(params, best_rmse)

In [7]:
def gridsearch(xtrain, params, params_grid):
    cols = ['boosting-round', 'test-rmse-mean', 'test-rmse-std', 'train-rmse-mean', 'train-rmse-std']
    cols.extend(sorted(params_grid.keys()))
    result = pd.DataFrame(columns=cols)

    count = 1
    combinations = get_params_combinations(params_grid)

    for param_overrides in combinations:
        print("Working on combination {}/{}".format(count, len(combinations)))
        count += 1
        params.update(param_overrides)
        tmp = xgb.cv(params, xtrain, nfold=5, num_boost_round=200, early_stopping_rounds=10, verbose_eval=10)
        tmp['boosting-round'] = tmp.index
        for param in param_overrides.keys():
            tmp[param] = param_overrides[param]
        result = result.append(tmp)

    print("Grid search ready!\n")

    return(result)

In [8]:
def ms2pip_pearson(true, pred):
    """
    Return pearson of tic-normalized, log-transformed intensities, 
    the MS2PIP way.
    """
    #tic_norm = lambda x: x / np.sum(x)
    # log_transform = lambda x: np.log2(x + 0.001)
    corr = pearsonr(
        true, 
        pred
    )[0]
    return (corr)

In [9]:
def spectral_angle(true, pred, epsilon=1e-7):
    """
    Return square root normalized spectral angle.
    See https://doi.org/10.1074/mcp.O113.036475
    """
    
    de_log = lambda x: (2**x)-0.001
    l2_normalize = lambda x: x / sqrt(max(sum(x**2), epsilon))
    
    pred_norm = l2_normalize(de_log(pred))
    true_norm = l2_normalize(de_log(true))
    
    spectral_angle = 1 - (2 * acos(np.dot(pred_norm, true_norm)) / np.pi)

    return (spectral_angle)

# New scoring method

In [10]:
class Scorer:
    def __init__(self,psmids):
        self.psmids = psmids
        
    def psm_score(self,targets, predictions):
        tmp = pd.DataFrame(columns=["psmids", "targets", "predictions"])
        tmp["psmids"] = np.array(self.psmids)
        tmp["targets"] = np.array(targets)
        tmp["predictions"] = np.array(predictions)
        tmp2 = tmp.groupby("psmids").agg({'predictions': list, 'targets': list}).reset_index()
        spectral_corr = []
        pearson_corr = []
        for spectra in range(0, len(tmp2["psmids"])):
            spectral_corr.append(spectral_angle(np.array(tmp2.targets.loc[spectra]), np.array(tmp2.predictions.loc[spectra])))
            pearson_corr.append(ms2pip_pearson(np.array(tmp2.targets.loc[spectra]), np.array(tmp2.predictions.loc[spectra])))
        return (pearson_corr, spectral_corr) 

# Hyperparameter optimization Y ions

In [11]:
vectors, targets, psmids = load_data("data/immunopeptide_chymotrypsin.pkl", "Y")

In [12]:
pd.Series(psmids.unique()).str.contains('PXD010154').sum()

60388

In [13]:
upeps = psmids.unique()
np.random.shuffle(upeps)
test_psms = upeps[:int(len(upeps) * 0.1)]

train_vectors = vectors[~psmids.isin(test_psms)]
train_targets = targets[~psmids.isin(test_psms)]
train_psmids = psmids[~psmids.isin(test_psms)]

test_vectors = vectors[psmids.isin(test_psms)]
test_targets = targets[psmids.isin(test_psms)]
test_psmids = psmids[psmids.isin(test_psms)]

In [14]:
pd.Series(test_psmids.unique()).str.contains('PXD010154').sum()

6036

In [15]:
pd.Series(train_psmids.unique()).str.contains('PXD010154').sum()

54352

In [16]:
xtrain = xgb.DMatrix(train_vectors, label=train_targets)
xtest = xgb.DMatrix(test_vectors, label=test_targets)
evallist = [(xtrain, 'train'),(xtest, 'test')]

In [18]:
space= {
    'eta': hp.loguniform('eta', np.log(0.01), np.log(1)),
    'max_depth': hp.quniform('max_depth', 3, 18, 1),
    'max_leaves': hp.quniform('max_leaves', 5, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'gamma' : hp.uniform ('gamma', 0.0,1),
    'min_child_weight' : hp.quniform('min_child_weight', 0, 500, 1),
    'subsample': hp.quniform('subsample', 0.5, 1 ,0.1),
    'reg_alpha': hp.quniform('reg_alpha', 0, 5 ,0.1)
}

In [179]:
def objective(space):
    params = {
    "nthread": 64,
    "objective": "reg:squarederror",
    "eval_metric": 'rmse',
    "eta": space["eta"],
    "max_depth": int(space['max_depth']),
    "grow_policy":"lossguide",
    "max_leaves":int(space["max_leaves"]),
    "min_child_weight": int(space["min_child_weight"]),
    "gamma": space['gamma'],
    "reg_lambda" : space['reg_lambda'],
    "colsample_bytree": space['colsample_bytree'],
    "subsample" : space["subsample"],
    "reg_alpha" : space["reg_alpha"]
    }
    print(params)
    tmp = xgb.cv(params, xtrain, nfold=4, num_boost_round=400, early_stopping_rounds=10, verbose_eval=True)
    rmse = tmp['test-rmse-mean'].min()

    print ("rmse:", rmse)
    return {'loss': rmse, 'status': STATUS_OK, "params": params}

In [180]:
trials = Trials()
best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 25,
                        trials = trials)


best_hyperparams

{'nthread': 64, 'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'eta': 0.5594367836426347, 'max_depth': 14, 'grow_policy': 'lossguide', 'max_leaves': 34, 'min_child_weight': 45, 'gamma': 0.7477872841305034, 'reg_lambda': 0.44407912770764824, 'colsample_bytree': 0.7049198028890877, 'subsample': 0.7000000000000001, 'reg_alpha': 1.3}
[0]	train-rmse:4.08630+0.00327	test-rmse:4.08739+0.00454

[1]	train-rmse:2.05432+0.00811	test-rmse:2.05882+0.00812

[2]	train-rmse:1.30230+0.00470	test-rmse:1.31318+0.00533

[3]	train-rmse:1.06907+0.00310	test-rmse:1.08691+0.00361

[4]	train-rmse:0.99899+0.00211	test-rmse:1.02267+0.00303

[5]	train-rmse:0.97418+0.00435	test-rmse:1.00265+0.00496

[6]	train-rmse:0.95905+0.00223	test-rmse:0.99162+0.00308

[7]	train-rmse:0.94691+0.00368	test-rmse:0.98297+0.00471

[8]	train-rmse:0.93786+0.00384	test-rmse:0.97781+0.00419

[9]	train-rmse:0.92964+0.00298	test-rmse:0.97305+0.00376

[10]	train-rmse:0.92249+0.00345	test-rmse:0.96900+0.00437

[11]	train-rmse:0.91

[135]	train-rmse:0.73921+0.00200	test-rmse:0.85335+0.00132                             

[136]	train-rmse:0.73832+0.00198	test-rmse:0.85308+0.00129                             

[137]	train-rmse:0.73743+0.00214	test-rmse:0.85280+0.00127                             

[138]	train-rmse:0.73657+0.00211	test-rmse:0.85254+0.00127                             

[139]	train-rmse:0.73570+0.00212	test-rmse:0.85228+0.00124                             

[140]	train-rmse:0.73469+0.00203	test-rmse:0.85193+0.00127                             

[141]	train-rmse:0.73383+0.00201	test-rmse:0.85165+0.00123                             

[142]	train-rmse:0.73282+0.00190	test-rmse:0.85125+0.00114                             

[143]	train-rmse:0.73207+0.00188	test-rmse:0.85105+0.00119                             

[144]	train-rmse:0.73134+0.00211	test-rmse:0.85079+0.00128                             

[145]	train-rmse:0.73059+0.00203	test-rmse:0.85058+0.00126                             

[146]	train-rmse:0.72

[319]	train-rmse:0.63907+0.00093	test-rmse:0.82953+0.00093                             

[320]	train-rmse:0.63861+0.00096	test-rmse:0.82946+0.00094                             

[321]	train-rmse:0.63815+0.00087	test-rmse:0.82940+0.00095                             

[322]	train-rmse:0.63778+0.00080	test-rmse:0.82935+0.00095                             

[323]	train-rmse:0.63732+0.00077	test-rmse:0.82929+0.00095                             

[324]	train-rmse:0.63691+0.00083	test-rmse:0.82923+0.00094                             

[325]	train-rmse:0.63645+0.00082	test-rmse:0.82915+0.00098                             

[326]	train-rmse:0.63614+0.00088	test-rmse:0.82911+0.00098                             

[327]	train-rmse:0.63573+0.00077	test-rmse:0.82905+0.00093                             

[328]	train-rmse:0.63518+0.00072	test-rmse:0.82897+0.00094                             

[329]	train-rmse:0.63489+0.00079	test-rmse:0.82892+0.00095                             

[330]	train-rmse:0.63

[107]	train-rmse:1.92543+0.00063	test-rmse:1.92916+0.00148                    

[108]	train-rmse:1.90342+0.00062	test-rmse:1.90722+0.00143                    

[109]	train-rmse:1.88190+0.00058	test-rmse:1.88577+0.00142                    

[110]	train-rmse:1.86076+0.00061	test-rmse:1.86471+0.00141                    

[111]	train-rmse:1.84008+0.00058	test-rmse:1.84408+0.00148                    

[112]	train-rmse:1.81972+0.00060	test-rmse:1.82382+0.00147                    

[113]	train-rmse:1.79983+0.00060	test-rmse:1.80398+0.00149                    

[114]	train-rmse:1.78028+0.00060	test-rmse:1.78451+0.00146                    

[115]	train-rmse:1.76111+0.00056	test-rmse:1.76543+0.00149                    

[116]	train-rmse:1.74234+0.00055	test-rmse:1.74674+0.00148                    

[117]	train-rmse:1.72396+0.00055	test-rmse:1.72843+0.00145                    

[118]	train-rmse:1.70587+0.00057	test-rmse:1.71041+0.00144                    

[119]	train-rmse:1.68821+0.00055	test-rm

[311]	train-rmse:0.91116+0.00044	test-rmse:0.92939+0.00117                    

[312]	train-rmse:0.91083+0.00043	test-rmse:0.92911+0.00118                    

[313]	train-rmse:0.91052+0.00041	test-rmse:0.92885+0.00118                    

[314]	train-rmse:0.91023+0.00044	test-rmse:0.92860+0.00116                    

[315]	train-rmse:0.90999+0.00045	test-rmse:0.92840+0.00114                    

[316]	train-rmse:0.90968+0.00047	test-rmse:0.92814+0.00113                    

[317]	train-rmse:0.90941+0.00044	test-rmse:0.92791+0.00116                    

[318]	train-rmse:0.90912+0.00044	test-rmse:0.92766+0.00115                    

[319]	train-rmse:0.90885+0.00044	test-rmse:0.92743+0.00114                    

[320]	train-rmse:0.90856+0.00039	test-rmse:0.92718+0.00117                    

[321]	train-rmse:0.90829+0.00038	test-rmse:0.92695+0.00118                    

[322]	train-rmse:0.90797+0.00041	test-rmse:0.92667+0.00117                    

[323]	train-rmse:0.90770+0.00038	test-rm

...

[161]	train-rmse:0.77994+0.00017	test-rmse:0.86225+0.00134                      

[162]	train-rmse:0.77927+0.00015	test-rmse:0.86196+0.00140                      

[163]	train-rmse:0.77864+0.00011	test-rmse:0.86171+0.00141                      

[164]	train-rmse:0.77801+0.00013	test-rmse:0.86147+0.00142                      

[165]	train-rmse:0.77741+0.00016	test-rmse:0.86120+0.00138                      

[166]	train-rmse:0.77682+0.00012	test-rmse:0.86094+0.00139                      

[167]	train-rmse:0.77608+0.00007	test-rmse:0.86062+0.00142                      

[168]	train-rmse:0.77540+0.00010	test-rmse:0.86035+0.00135                      

[169]	train-rmse:0.77481+0.00018	test-rmse:0.86014+0.00131                      

[170]	train-rmse:0.77418+0.00019	test-rmse:0.85992+0.00131                      

[171]	train-rmse:0.77347+0.00037	test-rmse:0.85961+0.00132                      

[172]	train-rmse:0.77294+0.00053	test-rmse:0.85939+0.00131                      

[173]	train-rmse

[359]	train-rmse:0.69677+0.00064	test-rmse:0.83535+0.00127                      

[360]	train-rmse:0.69647+0.00067	test-rmse:0.83527+0.00126                      

[361]	train-rmse:0.69622+0.00064	test-rmse:0.83521+0.00126                      

[362]	train-rmse:0.69597+0.00063	test-rmse:0.83514+0.00126                      

[363]	train-rmse:0.69569+0.00071	test-rmse:0.83507+0.00129                      

[364]	train-rmse:0.69544+0.00081	test-rmse:0.83500+0.00127                      

[365]	train-rmse:0.69517+0.00080	test-rmse:0.83494+0.00127                      

[366]	train-rmse:0.69483+0.00079	test-rmse:0.83487+0.00128                      

[367]	train-rmse:0.69462+0.00082	test-rmse:0.83482+0.00126                      

[368]	train-rmse:0.69435+0.00086	test-rmse:0.83476+0.00125                      

[369]	train-rmse:0.69409+0.00087	test-rmse:0.83471+0.00125                      

[370]	train-rmse:0.69372+0.00097	test-rmse:0.83460+0.00121                      

[371]	train-rmse

[151]	train-rmse:1.89748+0.00035	test-rmse:1.90394+0.00126                      

[152]	train-rmse:1.88175+0.00035	test-rmse:1.88829+0.00130                      

[153]	train-rmse:1.86624+0.00034	test-rmse:1.87287+0.00130                      

[154]	train-rmse:1.85093+0.00038	test-rmse:1.85766+0.00133                      

[155]	train-rmse:1.83579+0.00032	test-rmse:1.84261+0.00138                      

[156]	train-rmse:1.82089+0.00032	test-rmse:1.82779+0.00141                      

[157]	train-rmse:1.80623+0.00041	test-rmse:1.81323+0.00149                      

[158]	train-rmse:1.79175+0.00042	test-rmse:1.79885+0.00151                      

[159]	train-rmse:1.77743+0.00043	test-rmse:1.78462+0.00153                      

[160]	train-rmse:1.76334+0.00038	test-rmse:1.77062+0.00152                      

[161]	train-rmse:1.74949+0.00043	test-rmse:1.75687+0.00158                      

[162]	train-rmse:1.73582+0.00039	test-rmse:1.74330+0.00155                      

[163]	train-rmse

[349]	train-rmse:0.90623+0.00058	test-rmse:0.93386+0.00115                      

[350]	train-rmse:0.90551+0.00054	test-rmse:0.93323+0.00117                      

[351]	train-rmse:0.90482+0.00054	test-rmse:0.93262+0.00114                      

[352]	train-rmse:0.90415+0.00054	test-rmse:0.93204+0.00111                      

[353]	train-rmse:0.90344+0.00052	test-rmse:0.93142+0.00115                      

[354]	train-rmse:0.90281+0.00050	test-rmse:0.93087+0.00119                      

[355]	train-rmse:0.90217+0.00047	test-rmse:0.93032+0.00120                      

[356]	train-rmse:0.90152+0.00047	test-rmse:0.92975+0.00117                      

[357]	train-rmse:0.90091+0.00048	test-rmse:0.92923+0.00116                      

[358]	train-rmse:0.90025+0.00048	test-rmse:0.92866+0.00116                      

[359]	train-rmse:0.89964+0.00047	test-rmse:0.92814+0.00116                      

[360]	train-rmse:0.89903+0.00048	test-rmse:0.92761+0.00121                      

[361]	train-rmse

{'colsample_bytree': 0.6122042447952851,
 'eta': 0.047107785048838,
 'gamma': 0.37528441949267444,
 'max_depth': 18.0,
 'max_leaves': 490.0,
 'min_child_weight': 4.0,
 'reg_alpha': 3.3000000000000003,
 'reg_lambda': 0.35150807248415,
 'subsample': 0.6000000000000001}

In [181]:
hyperoptimization_results = {'test-rmse-mean': [x['loss'] for x in trials.results]}
for key in trials.results[0]["params"].keys():
    hyperoptimization_results[key] = [x['params'][key] for x in trials.results]

In [182]:
df = pd.DataFrame(hyperoptimization_results)
df.head()

,test-rmse-mean,nthread,objective,eval_metric,eta,max_depth,grow_policy,max_leaves,min_child_weight,gamma,reg_lambda,colsample_bytree,subsample,reg_alpha
0,0.942563,64,reg:squarederror,rmse,0.559437,14,lossguide,34,45,0.747787,0.444079,0.704920,0.7,1.3
1,0.825310,64,reg:squarederror,rmse,0.097313,15,lossguide,386,35,0.518029,0.139861,0.581350,0.9,3.4
2,0.912834,64,reg:squarederror,rmse,0.015615,15,lossguide,426,479,0.090244,0.478151,0.707745,0.9,2.3
3,0.932483,64,reg:squarederror,rmse,0.021922,12,lossguide,14,459,0.609327,0.945877,0.753762,0.6,3.9
4,0.847209,64,reg:squarederror,rmse,0.077652,18,lossguide,256,380,0.492369,0.990319,0.837916,0.7,3.8


In [183]:
df.to_csv("data/immunopeptide_chymotrypsin_resultsY.csv")

In [191]:
params = {
    "nthread": 24,
    "objective": "reg:squarederror",
    #"objective": "binary:logistic",
    #"eval_metric": 'mae',
    "eval_metric": 'rmse',
    "eta": 0.5,
    "max_depth": 9,
    "grow_policy":"lossguide",
    "max_leaves":100,
    "min_child_weight": 300,
    "gamma": 0.1,
    "subsample": 1,
    "reg_lambda" : 0,
    "reg_alpha" : 0,
    "colsample_bytree": 1,
}

In [192]:
params.update(best_hyperparams)

In [193]:
params["max_depth"] = int(params["max_depth"])
params["max_leaves"] = int(params["max_leaves"])
params["min_child_weight"] = int(params["min_child_weight"])
params

{'nthread': 24,
 'objective': 'reg:squarederror',
 'eval_metric': 'rmse',
 'eta': 0.047107785048838,
 'max_depth': 18,
 'grow_policy': 'lossguide',
 'max_leaves': 490,
 'min_child_weight': 4,
 'gamma': 0.37528441949267444,
 'subsample': 0.6000000000000001,
 'reg_lambda': 0.35150807248415,
 'reg_alpha': 3.3000000000000003,
 'colsample_bytree': 0.6122042447952851}

In [17]:
params = {'nthread': 8,
 'objective': 'reg:squarederror',
 'eval_metric': 'rmse',
 'eta': 0.047107785048838,
 'max_depth': 18,
 'grow_policy': 'lossguide',
 'max_leaves': 490,
 'min_child_weight': 4,
 'gamma': 0.37528441949267444,
 'subsample': 0.6000000000000001,
 'reg_lambda': 0.35150807248415,
 'reg_alpha': 3.3000000000000003,
 'colsample_bytree': 0.6122042447952851}

In [ ]:
bst = xgb.train(params, xtrain, 400, evallist, maximize=False, early_stopping_rounds=10)
bst.save_model("data/Immunopeptide_chymotrypsin_HCDy.xgboost")


[0]	train-rmse:8.53439	test-rmse:8.52819
[1]	train-rmse:8.14145	test-rmse:8.13361
[2]	train-rmse:7.76612	test-rmse:7.75725
[3]	train-rmse:7.40262	test-rmse:7.39970
[4]	train-rmse:7.06180	test-rmse:7.05939
[5]	train-rmse:6.73789	test-rmse:6.73484
[6]	train-rmse:6.42800	test-rmse:6.42592
[7]	train-rmse:6.13145	test-rmse:6.13226
[8]	train-rmse:5.85177	test-rmse:5.85208
[9]	train-rmse:5.58802	test-rmse:5.58598
[10]	train-rmse:5.33276	test-rmse:5.33248
[11]	train-rmse:5.09077	test-rmse:5.09190
[12]	train-rmse:4.86173	test-rmse:4.86231
[13]	train-rmse:4.64311	test-rmse:4.64447
[14]	train-rmse:4.43775	test-rmse:4.43662
[15]	train-rmse:4.23750	test-rmse:4.23912
[16]	train-rmse:4.04983	test-rmse:4.05119
[17]	train-rmse:3.87075	test-rmse:3.87262
[18]	train-rmse:3.70185	test-rmse:3.70309
[19]	train-rmse:3.54035	test-rmse:3.54236
[20]	train-rmse:3.38629	test-rmse:3.38912
[21]	train-rmse:3.23936	test-rmse:3.24359
[22]	train-rmse:3.10211	test-rmse:3.10558
[23]	train-rmse:2.97045	test-rmse:2.97409
[2

[194]	train-rmse:0.65880	test-rmse:0.84284
[195]	train-rmse:0.65756	test-rmse:0.84262
[196]	train-rmse:0.65688	test-rmse:0.84251
[197]	train-rmse:0.65618	test-rmse:0.84239
[198]	train-rmse:0.65527	test-rmse:0.84225
[199]	train-rmse:0.65440	test-rmse:0.84209
[200]	train-rmse:0.65353	test-rmse:0.84202
[201]	train-rmse:0.65281	test-rmse:0.84190
[202]	train-rmse:0.65209	test-rmse:0.84178
[203]	train-rmse:0.65091	test-rmse:0.84157
[204]	train-rmse:0.65014	test-rmse:0.84144
[205]	train-rmse:0.64945	test-rmse:0.84130
[206]	train-rmse:0.64853	test-rmse:0.84118
[207]	train-rmse:0.64782	test-rmse:0.84111
[208]	train-rmse:0.64684	test-rmse:0.84099
[209]	train-rmse:0.64608	test-rmse:0.84088
[210]	train-rmse:0.64513	test-rmse:0.84073
[211]	train-rmse:0.64435	test-rmse:0.84062
[212]	train-rmse:0.64381	test-rmse:0.84052
[213]	train-rmse:0.64293	test-rmse:0.84039
[214]	train-rmse:0.64207	test-rmse:0.84031
[215]	train-rmse:0.64134	test-rmse:0.84014
[216]	train-rmse:0.64047	test-rmse:0.84001
[217]	train

[385]	train-rmse:0.54141	test-rmse:0.83062
[386]	train-rmse:0.54077	test-rmse:0.83060
[387]	train-rmse:0.54048	test-rmse:0.83058
[388]	train-rmse:0.53998	test-rmse:0.83056
[389]	train-rmse:0.53968	test-rmse:0.83056
[390]	train-rmse:0.53917	test-rmse:0.83056
[391]	train-rmse:0.53869	test-rmse:0.83055
[392]	train-rmse:0.53819	test-rmse:0.83056
[393]	train-rmse:0.53777	test-rmse:0.83054
[394]	train-rmse:0.53716	test-rmse:0.83055
[395]	train-rmse:0.53677	test-rmse:0.83054
[396]	train-rmse:0.53643	test-rmse:0.83053
[397]	train-rmse:0.53614	test-rmse:0.83051
[398]	train-rmse:0.53580	test-rmse:0.83049
[399]	train-rmse:0.53543	test-rmse:0.83048


In [ ]:
bst.save_model("Immunopeptide_chymotrypsin_HCDy(2).xgboost")

In [ ]:
print("statement")

In [ ]:
print("statement")

In [ ]:
print("statement")

In [ ]:
print("statement")

In [ ]:
print("statement")

In [ ]:
print("statement")

In [ ]:
print("statement")

In [ ]:
print("statement")

In [ ]:
print("statement")

In [ ]:
print("statement")

In [ ]:
print("statement")

In [ ]:
print("statement")

In [ ]:
print("statement")

# Fast y-ion model

In [ ]:
space= {
    'eta': hp.loguniform('eta', np.log(0.01), np.log(1)),
    'max_depth': hp.quniform('max_depth', 2, 7, 1),
    'max_leaves': hp.quniform('max_leaves', 5, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'gamma' : hp.uniform ('gamma', 0.0,1),
    'min_child_weight' : hp.quniform('min_child_weight', 0, 500, 1),
    'subsample': hp.quniform('subsample', 0.5, 1 ,0.1),
    'reg_alpha': hp.quniform('reg_alpha', 0, 5 ,0.1),
    'grow_policy': hp.choice('grow_policy', ("lossguide", "depthwise"))
}

In [ ]:
def objective(space):
    params = {
    "nthread": 64,
    "objective": "reg:squarederror",
    "eval_metric": 'rmse',
    "eta": space["eta"],
    "max_depth": int(space['max_depth']),
    "grow_policy":space['grow_policy'],
    "max_leaves":int(space["max_leaves"]),
    "min_child_weight": int(space["min_child_weight"]),
    "gamma": space['gamma'],
    "reg_lambda" : space['reg_lambda'],
    "colsample_bytree": space['colsample_bytree'],
    "subsample" : space["subsample"],
    "reg_alpha" : space["reg_alpha"]
    }
    print(params)
    tmp = xgb.cv(params, xtrain, nfold=4, num_boost_round=400, early_stopping_rounds=10, verbose_eval=True)
    rmse = tmp['test-rmse-mean'].min()

    print ("rmse:", rmse)
    return {'loss': rmse, 'status': STATUS_OK, "params": params}

In [198]:
trials = Trials()
best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 30,
                        trials = trials)


best_hyperparams

{'nthread': 64, 'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'eta': 0.021115437192212663, 'max_depth': 3, 'grow_policy': 'lossguide', 'max_leaves': 15, 'min_child_weight': 208, 'gamma': 0.12272183849629525, 'reg_lambda': 0.09879364587990913, 'colsample_bytree': 0.9768735793310527, 'subsample': 0.8, 'reg_alpha': 3.3000000000000003}
[0]	train-rmse:8.76231+0.00043	test-rmse:8.76246+0.00130

[1]	train-rmse:8.58372+0.00076	test-rmse:8.58391+0.00127

[2]	train-rmse:8.40962+0.00052	test-rmse:8.40948+0.00127

[3]	train-rmse:8.23926+0.00040	test-rmse:8.23855+0.00125

[4]	train-rmse:8.07077+0.00043	test-rmse:8.07152+0.00121

[5]	train-rmse:7.90871+0.00048	test-rmse:7.90824+0.00119

[6]	train-rmse:7.74896+0.00036	test-rmse:7.74854+0.00112

[7]	train-rmse:7.59215+0.00044	test-rmse:7.59230+0.00111

[8]	train-rmse:7.43933+0.00047	test-rmse:7.43961+0.00107

[9]	train-rmse:7.29061+0.00039	test-rmse:7.29037+0.00116

[10]	train-rmse:7.14446+0.00042	test-rmse:7.14448+0.00121

[11]	train-rmse:7

[268]	train-rmse:1.37833+0.00086	test-rmse:1.37844+0.00150

[269]	train-rmse:1.37781+0.00082	test-rmse:1.37792+0.00141

[270]	train-rmse:1.37734+0.00079	test-rmse:1.37745+0.00137

[271]	train-rmse:1.37683+0.00086	test-rmse:1.37695+0.00144

[272]	train-rmse:1.37624+0.00094	test-rmse:1.37635+0.00157

[273]	train-rmse:1.37585+0.00095	test-rmse:1.37596+0.00158

[274]	train-rmse:1.37539+0.00094	test-rmse:1.37550+0.00153

[275]	train-rmse:1.37499+0.00086	test-rmse:1.37510+0.00145

[276]	train-rmse:1.37432+0.00076	test-rmse:1.37443+0.00138

[277]	train-rmse:1.37386+0.00063	test-rmse:1.37397+0.00128

[278]	train-rmse:1.37340+0.00060	test-rmse:1.37351+0.00123

[279]	train-rmse:1.37304+0.00060	test-rmse:1.37315+0.00123

[280]	train-rmse:1.37263+0.00058	test-rmse:1.37273+0.00124

[281]	train-rmse:1.37200+0.00071	test-rmse:1.37210+0.00142

[282]	train-rmse:1.37136+0.00074	test-rmse:1.37147+0.00142

[283]	train-rmse:1.37093+0.00080	test-rmse:1.37104+0.00142

[284]	train-rmse:1.37048+0.00085	test-rm

[0]	train-rmse:7.70736+0.00043	test-rmse:7.70690+0.00114                       

[1]	train-rmse:6.65610+0.00024	test-rmse:6.65588+0.00127                       

[2]	train-rmse:5.76145+0.00170	test-rmse:5.76155+0.00143                       

[3]	train-rmse:5.00370+0.00125	test-rmse:5.00364+0.00229                       

[4]	train-rmse:4.36189+0.00094	test-rmse:4.36189+0.00182                       

[5]	train-rmse:3.82312+0.00111	test-rmse:3.82312+0.00202                       

[6]	train-rmse:3.37177+0.00059	test-rmse:3.37181+0.00168                       

[7]	train-rmse:2.99566+0.00040	test-rmse:2.99573+0.00155                       

[8]	train-rmse:2.68504+0.00057	test-rmse:2.68508+0.00168                       

[9]	train-rmse:2.43031+0.00160	test-rmse:2.43045+0.00247                       

[10]	train-rmse:2.22396+0.00163	test-rmse:2.22409+0.00221                      

[11]	train-rmse:2.05769+0.00138	test-rmse:2.05782+0.00195                      

[12]	train-rmse:1.92512+0.00

[202]	train-rmse:1.20448+0.00193	test-rmse:1.20475+0.00128                     

[203]	train-rmse:1.20386+0.00219	test-rmse:1.20412+0.00149                     

[204]	train-rmse:1.20356+0.00216	test-rmse:1.20382+0.00142                     

[205]	train-rmse:1.20327+0.00226	test-rmse:1.20353+0.00151                     

[206]	train-rmse:1.20290+0.00207	test-rmse:1.20316+0.00129                     

[207]	train-rmse:1.20256+0.00210	test-rmse:1.20281+0.00130                     

[208]	train-rmse:1.20219+0.00224	test-rmse:1.20245+0.00147                     

[209]	train-rmse:1.20178+0.00255	test-rmse:1.20203+0.00175                     

[210]	train-rmse:1.20138+0.00252	test-rmse:1.20164+0.00169                     

[211]	train-rmse:1.20096+0.00255	test-rmse:1.20121+0.00172                     

[212]	train-rmse:1.20052+0.00258	test-rmse:1.20077+0.00173                     

[213]	train-rmse:1.20009+0.00274	test-rmse:1.20035+0.00189                     

[214]	train-rmse:1.19935+0.0

[0]	train-rmse:5.62920+0.00049	test-rmse:5.62928+0.00108                      

[1]	train-rmse:3.64909+0.00162	test-rmse:3.64916+0.00138                      

[2]	train-rmse:2.49887+0.00402	test-rmse:2.49903+0.00484                      

[3]	train-rmse:1.87466+0.00713	test-rmse:1.87491+0.00787                      

[4]	train-rmse:1.55420+0.01054	test-rmse:1.55451+0.01163                      

[5]	train-rmse:1.40077+0.01428	test-rmse:1.40120+0.01533                      

[6]	train-rmse:1.32490+0.00912	test-rmse:1.32536+0.01012                      

[7]	train-rmse:1.28158+0.00696	test-rmse:1.28202+0.00788                      

[8]	train-rmse:1.25723+0.00865	test-rmse:1.25765+0.00929                      

[9]	train-rmse:1.23413+0.00670	test-rmse:1.23465+0.00729                      

[10]	train-rmse:1.22215+0.00607	test-rmse:1.22274+0.00650                     

[11]	train-rmse:1.21080+0.00640	test-rmse:1.21147+0.00689                     

[12]	train-rmse:1.19748+0.00668	test-rms

...

[276]	train-rmse:0.94192+0.00186	test-rmse:0.95113+0.00280                      

[277]	train-rmse:0.94171+0.00182	test-rmse:0.95095+0.00278                      

[278]	train-rmse:0.94155+0.00180	test-rmse:0.95082+0.00276                      

[279]	train-rmse:0.94139+0.00187	test-rmse:0.95068+0.00281                      

[280]	train-rmse:0.94119+0.00190	test-rmse:0.95051+0.00289                      

[281]	train-rmse:0.94101+0.00190	test-rmse:0.95036+0.00289                      

[282]	train-rmse:0.94083+0.00191	test-rmse:0.95020+0.00291                      

[283]	train-rmse:0.94069+0.00190	test-rmse:0.95009+0.00290                      

[284]	train-rmse:0.94051+0.00187	test-rmse:0.94995+0.00286                      

[285]	train-rmse:0.94029+0.00187	test-rmse:0.94976+0.00284                      

[286]	train-rmse:0.94018+0.00188	test-rmse:0.94968+0.00286                      

[287]	train-rmse:0.93992+0.00195	test-rmse:0.94945+0.00295                      

[288]	train-rmse

[68]	train-rmse:1.03483+0.00277	test-rmse:1.04625+0.00279                       

[69]	train-rmse:1.03392+0.00270	test-rmse:1.04547+0.00269                       

[70]	train-rmse:1.03308+0.00258	test-rmse:1.04476+0.00261                       

[71]	train-rmse:1.03227+0.00262	test-rmse:1.04410+0.00256                       

[72]	train-rmse:1.03164+0.00272	test-rmse:1.04363+0.00275                       

[73]	train-rmse:1.03054+0.00263	test-rmse:1.04265+0.00274                       

[74]	train-rmse:1.02978+0.00247	test-rmse:1.04210+0.00256                       

[75]	train-rmse:1.02895+0.00232	test-rmse:1.04140+0.00249                       

[76]	train-rmse:1.02828+0.00210	test-rmse:1.04090+0.00234                       

[77]	train-rmse:1.02752+0.00226	test-rmse:1.04031+0.00251                       

[78]	train-rmse:1.02657+0.00215	test-rmse:1.03951+0.00236                       

[79]	train-rmse:1.02553+0.00238	test-rmse:1.03859+0.00263                       

[80]	train-rmse:

[266]	train-rmse:0.94831+0.00089	test-rmse:0.98728+0.00119                      

[267]	train-rmse:0.94809+0.00087	test-rmse:0.98719+0.00121                      

[268]	train-rmse:0.94773+0.00092	test-rmse:0.98697+0.00140                      

[269]	train-rmse:0.94749+0.00097	test-rmse:0.98687+0.00144                      

[270]	train-rmse:0.94728+0.00099	test-rmse:0.98680+0.00146                      

[271]	train-rmse:0.94706+0.00101	test-rmse:0.98672+0.00146                      

[272]	train-rmse:0.94680+0.00100	test-rmse:0.98660+0.00143                      

[273]	train-rmse:0.94656+0.00101	test-rmse:0.98650+0.00142                      

[274]	train-rmse:0.94637+0.00103	test-rmse:0.98644+0.00144                      

[275]	train-rmse:0.94612+0.00102	test-rmse:0.98631+0.00151                      

[276]	train-rmse:0.94587+0.00102	test-rmse:0.98620+0.00154                      

[277]	train-rmse:0.94566+0.00099	test-rmse:0.98613+0.00153                      

[278]	train-rmse

[58]	train-rmse:1.05663+0.00275	test-rmse:1.05891+0.00236                       

[59]	train-rmse:1.05505+0.00221	test-rmse:1.05736+0.00194                       

[60]	train-rmse:1.05391+0.00202	test-rmse:1.05625+0.00185                       

[61]	train-rmse:1.05246+0.00223	test-rmse:1.05482+0.00187                       

[62]	train-rmse:1.05145+0.00231	test-rmse:1.05384+0.00199                       

[63]	train-rmse:1.04999+0.00229	test-rmse:1.05242+0.00210                       

[64]	train-rmse:1.04840+0.00170	test-rmse:1.05088+0.00148                       

[65]	train-rmse:1.04741+0.00172	test-rmse:1.04991+0.00155                       

[66]	train-rmse:1.04621+0.00147	test-rmse:1.04876+0.00132                       

[67]	train-rmse:1.04468+0.00145	test-rmse:1.04727+0.00159                       

[68]	train-rmse:1.04399+0.00143	test-rmse:1.04663+0.00166                       

[69]	train-rmse:1.04302+0.00117	test-rmse:1.04568+0.00154                       

[70]	train-rmse:

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [199]:
best_hyperparams

{'colsample_bytree': 0.7223765548301422,
 'eta': 0.43840677476149614,
 'gamma': 0.0642274935539753,
 'grow_policy': 1,
 'max_depth': 7.0,
 'max_leaves': 97.0,
 'min_child_weight': 76.0,
 'reg_alpha': 1.4000000000000001,
 'reg_lambda': 0.36282696421401306,
 'subsample': 0.9}

In [200]:
hyperoptimization_results = {'test-rmse-mean': [x['loss'] for x in trials.results]}
for key in trials.results[0]["params"].keys():
    hyperoptimization_results[key] = [x['params'][key] for x in trials.results]

In [201]:
df = pd.DataFrame(hyperoptimization_results)

In [202]:
df

,test-rmse-mean,nthread,objective,eval_metric,eta,max_depth,grow_policy,max_leaves,min_child_weight,gamma,reg_lambda,colsample_bytree,subsample,reg_alpha
0,1.326238,64,reg:squarederror,rmse,0.021115,3,lossguide,15,208,0.122722,0.098794,0.976874,0.8,3.3
1,1.151331,64,reg:squarederror,rmse,0.143571,3,depthwise,329,237,0.034293,0.061191,0.918636,0.5,2.4
2,0.955761,64,reg:squarederror,rmse,0.384498,6,depthwise,82,99,0.020850,0.577328,0.584920,0.6,1.2
3,1.035049,64,reg:squarederror,rmse,0.113717,5,lossguide,462,220,0.274894,0.015945,0.948690,0.5,1.2
4,1.305909,64,reg:squarederror,rmse,0.079629,2,lossguide,470,12,0.865298,0.692385,0.573204,0.7,3.6
5,1.308287,64,reg:squarederror,rmse,0.013231,4,lossguide,376,254,0.817714,0.118411,0.839755,0.9,3.2
6,1.096536,64,reg:squarederror,rmse,0.319268,3,depthwise,173,268,0.339701,0.036753,0.761881,0.8,3.6
7,1.075102,64,reg:squarederror,rmse,0.031466,6,depthwise,359,28,0.200102,0.877922,0.952982,0.8,2.1
8,1.100769,64,reg:squarederror,rmse,0.310908,3,lossguide,157,486,0.865972,0.456516,0.608405,1.0,3.7
9,1.142858,64,reg:squarederror,rmse,0.061756,4,depthwise,166,358,0.873785,0.216095,0.951954,0.5,0.2


In [203]:
df.to_csv("data/immunopeptide_chymotrypsin_resultsY.csv", mode="a", header=False)


In [204]:
params = {
    "nthread": 64,
    "objective": "reg:squarederror",
    #"objective": "binary:logistic",
    #"eval_metric": 'mae',
    "eval_metric": 'rmse',
    "eta": 0.5,
    "max_depth": 9,
    "grow_policy":"lossguide",
    "max_leaves":100,
    "min_child_weight": 300,
    "gamma": 0.1,
    "subsample": 1,
    "reg_lambda" : 0,
    "colsample_bytree": 1,
    "reg_alpha" : 0
}

In [205]:
best_hyperparams = space_eval(space, best_hyperparams)
params.update(best_hyperparams)

In [206]:
params["max_depth"] = int(params["max_depth"])
params["max_leaves"] = int(params["max_leaves"])
params["min_child_weight"] = int(params["min_child_weight"])


In [207]:
params

{'nthread': 64,
 'objective': 'reg:squarederror',
 'eval_metric': 'rmse',
 'eta': 0.43840677476149614,
 'max_depth': 7,
 'grow_policy': 'depthwise',
 'max_leaves': 97,
 'min_child_weight': 76,
 'gamma': 0.0642274935539753,
 'subsample': 0.9,
 'reg_lambda': 0.36282696421401306,
 'colsample_bytree': 0.7223765548301422,
 'reg_alpha': 1.4000000000000001}

In [208]:
bst2 = xgb.train(params, xtrain, 400, evallist, maximize=False, early_stopping_rounds=10)

[0]	train-rmse:5.15910	test-rmse:5.15346
[1]	train-rmse:3.11141	test-rmse:3.10747
[2]	train-rmse:2.05661	test-rmse:2.05352
[3]	train-rmse:1.55862	test-rmse:1.55671
[4]	train-rmse:1.34618	test-rmse:1.34596
[5]	train-rmse:1.26211	test-rmse:1.26280
[6]	train-rmse:1.22342	test-rmse:1.22479
[7]	train-rmse:1.20062	test-rmse:1.20247
[8]	train-rmse:1.18363	test-rmse:1.18550
[9]	train-rmse:1.16759	test-rmse:1.16969
[10]	train-rmse:1.16019	test-rmse:1.16228
[11]	train-rmse:1.14686	test-rmse:1.14899
[12]	train-rmse:1.13577	test-rmse:1.13798
[13]	train-rmse:1.12704	test-rmse:1.12941
[14]	train-rmse:1.11864	test-rmse:1.12134
[15]	train-rmse:1.11183	test-rmse:1.11461
[16]	train-rmse:1.10654	test-rmse:1.10944
[17]	train-rmse:1.10206	test-rmse:1.10510
[18]	train-rmse:1.09870	test-rmse:1.10167
[19]	train-rmse:1.09419	test-rmse:1.09724
[20]	train-rmse:1.08954	test-rmse:1.09259
[21]	train-rmse:1.08318	test-rmse:1.08628
[22]	train-rmse:1.08159	test-rmse:1.08480
[23]	train-rmse:1.07869	test-rmse:1.08203
[2

[194]	train-rmse:0.94417	test-rmse:0.95800
[195]	train-rmse:0.94403	test-rmse:0.95793
[196]	train-rmse:0.94370	test-rmse:0.95769
[197]	train-rmse:0.94356	test-rmse:0.95759
[198]	train-rmse:0.94331	test-rmse:0.95744
[199]	train-rmse:0.94299	test-rmse:0.95722
[200]	train-rmse:0.94290	test-rmse:0.95714
[201]	train-rmse:0.94271	test-rmse:0.95696
[202]	train-rmse:0.94239	test-rmse:0.95671
[203]	train-rmse:0.94213	test-rmse:0.95650
[204]	train-rmse:0.94186	test-rmse:0.95628
[205]	train-rmse:0.94155	test-rmse:0.95600
[206]	train-rmse:0.94132	test-rmse:0.95581
[207]	train-rmse:0.94107	test-rmse:0.95563
[208]	train-rmse:0.94062	test-rmse:0.95524
[209]	train-rmse:0.94047	test-rmse:0.95514
[210]	train-rmse:0.94015	test-rmse:0.95492
[211]	train-rmse:0.93999	test-rmse:0.95482
[212]	train-rmse:0.93938	test-rmse:0.95429
[213]	train-rmse:0.93922	test-rmse:0.95420
[214]	train-rmse:0.93875	test-rmse:0.95375
[215]	train-rmse:0.93855	test-rmse:0.95361
[216]	train-rmse:0.93837	test-rmse:0.95350
[217]	train

[385]	train-rmse:0.90768	test-rmse:0.93236
[386]	train-rmse:0.90752	test-rmse:0.93227
[387]	train-rmse:0.90739	test-rmse:0.93215
[388]	train-rmse:0.90716	test-rmse:0.93201
[389]	train-rmse:0.90698	test-rmse:0.93186
[390]	train-rmse:0.90679	test-rmse:0.93174
[391]	train-rmse:0.90660	test-rmse:0.93163
[392]	train-rmse:0.90646	test-rmse:0.93155
[393]	train-rmse:0.90629	test-rmse:0.93144
[394]	train-rmse:0.90619	test-rmse:0.93138
[395]	train-rmse:0.90610	test-rmse:0.93132
[396]	train-rmse:0.90594	test-rmse:0.93121
[397]	train-rmse:0.90572	test-rmse:0.93105
[398]	train-rmse:0.90554	test-rmse:0.93095
[399]	train-rmse:0.90546	test-rmse:0.93092


In [209]:
bst2.save_model("Models/Fast_immunopeptide_chymotrypsin_HCDy.xgboost")